This is a kfold with both xgb and lgb. Some elements are from kernels by The1owl and HyungsukKang.


참고 - https://www.kaggle.com/rshally/porto-xgb-lgb-kfold-lb-0-282

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
import gc

print('loading files...')
train = pd.read_csv('../input/train.csv', na_values=-1, nrows = 10000)
test = pd.read_csv('../input/test.csv', na_values=-1, nrows = 10000)
col_to_drop = train.columns[train.columns.str.startswith('ps_calc_')]
train = train.drop(col_to_drop, axis=1)  
test = test.drop(col_to_drop, axis=1)  

for c in train.select_dtypes(include=['float64']).columns:
    train[c]=train[c].astype(np.float32)
    test[c]=test[c].astype(np.float32)
for c in train.select_dtypes(include=['int64']).columns[2:]:
    train[c]=train[c].astype(np.int8)
    test[c]=test[c].astype(np.int8)    

print(train.shape, test.shape)

In [ ]:
# custom objective function (similar to auc)

def gini(y, pred):
    g = np.asarray(np.c_[y, pred, np.arange(len(y)) ], dtype=np.float)
    g = g[np.lexsort((g[:,2], -1*g[:,1]))]
    gs = g[:,0].cumsum().sum() / g[:,0].sum()
    gs -= (len(y) + 1) / 2.
    return gs / len(y)

def gini_xgb(pred, y):
    y = y.get_label()
    return 'gini', gini(y, pred) / gini(y, y)

def gini_lgb(preds, dtrain):
    y = list(dtrain.get_label())
    score = gini(y, preds) / gini(y, y)
    return 'gini', score, True

In [ ]:
# xgb
# https://xgboost.readthedocs.io/en/latest/parameter.html#
#https://lightgbm.readthedocs.io/en/latest/Parameters.html
# 파라미터의 의미를 알려면 모델(알고리즘)을 공부해야한다.
params = {'eta': 0.02,
          'max_depth': 4,
          'subsample': 0.9,
          'colsample_bytree': 0.9, 
          'objective': 'binary:logistic',
          'eval_metric': 'auc'}
#           'silent': True}

In [ ]:
X = train.drop(['id', 'target'], axis=1)

In [ ]:
features = X.columns

In [ ]:
X = X.values

In [ ]:
y = train['target'].values

In [ ]:
X

In [ ]:
y

In [ ]:
# Stacking을 하기 위해 학습할 것과 Submission할것을 준비해 놓는다.
sub = test['id'].to_frame()

In [ ]:
sub

In [ ]:
sub['target'] = 0

In [ ]:
sub

In [ ]:
nrounds=200  # need to change to 2000
kfold = 2  # need to change to 5
skf = StratifiedKFold(n_splits=kfold, random_state=0)
# 중요. Binary Classification에서 Dataset이 Imbalance하기때문에 Stratified해야함.
# Fold를 나눠도 0, 1의 비율을 Fold마다 맞춰준다.

In [ ]:
for i, (train_index, valid_index) in enumerate(skf.split(X, y)): # Dataset과 y를 넣어줘야 어떻게 나눌지 볼수 있으므로...
    print(' xgb kfold: {}  of  {} : '.format(i+1, kfold))

    break;


In [ ]:
train_index

In [ ]:
valid_index

In [ ]:
y.mean()

In [ ]:
train_index

In [ ]:
y[train_index].mean()

In [ ]:
X_train, X_valid = X[train_index], X[valid_index]

In [ ]:
y_train, y_valid = y[train_index], y[valid_index]

In [ ]:
# DMatrix를 만들어 줘야함. 학습할 때 빨리 진행되도록
d_train = xgb.DMatrix(X_train, y_train) # 학습할 데이터셋과 레이블을 인자로..
d_valid = xgb.DMatrix(X_valid, y_valid) 

In [ ]:
watchlist = [(d_train, 'train'), (d_valid, 'valid')] 
# 학습이 얼마나 진행/잘되고 있는지 봐야함. --> 중지시켜야함.

In [ ]:
# 부스팅은 여기에 sequential이 추가된다. 
#즉 연속적인 weak learner, 바로 직전 weak learner의 error를 반영한 현재 weak learner를 잡겠다는 아이디어이다.
# Score: AUC          'eval_metric': 'auc',
xgb_model = xgb.train(params, 
          d_train, 
          nrounds, # adaptive하게 추가함으로 round를 넣어줌.
          watchlist, 
          early_stopping_rounds=100) # 매 라운드마다 Score를 보다가 100개가 되었는데도 스코어가 좋아지지 않으면 끊음.
          #feval=gini_xgb, 
          #maximize=True, 
          #verbose_eval=100)

In [ ]:
xgb_model = xgb.train(params, 
          d_train, 
          nrounds, # adaptive하게 추가함으로 round를 넣어줌.
          watchlist, 
          early_stopping_rounds=100, # 매 라운드마다 Score를 보다가 100개가 되었는데도 스코어가 좋아지지 않으면 끊음.
          feval=gini_xgb, # GINI계수를 사용함.
          maximize=True, 
          verbose_eval=100)

In [ ]:
xgb_model.best_ntree_limit # 모델학습한 후 가장 베스트 interation이 남아있다.

In [ ]:
sub['target'] += xgb_model.predict(xgb.DMatrix(test[features].values), 
                        ntree_limit=xgb_model.best_ntree_limit+50) / (2*kfold)
# ntree_limit: predict할때 언제 끊어라
# 2*kfold --> xboost를 2개의 폴드로 나눠서 모델 2개로. 0.5를 채우기위해 1/4씩(반복할때마다...)
# lgbm 도 2*kfold로 나눠서 1/4
# --> xboost + lbgm = 1
# Staking은 기본이다.. Single model 보다 더 좋은 결과를 일단 얻을수있다. 기본으로 써라!!!

In [ ]:
sub.head(2)